In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kavsakdata/SS/screenshot_16_162507.png
/kaggle/input/kavsakdata/SS/screenshot_15_223843.png
/kaggle/input/kavsakdata/SS/screenshot_15_232238.png
/kaggle/input/kavsakdata/SS/screenshot_16_010612.png
/kaggle/input/kavsakdata/SS/screenshot_16_014127.png
/kaggle/input/kavsakdata/SS/screenshot_16_094751.png
/kaggle/input/kavsakdata/SS/screenshot_16_212026.png
/kaggle/input/kavsakdata/SS/screenshot_16_012224.png
/kaggle/input/kavsakdata/SS/screenshot_16_142736.png
/kaggle/input/kavsakdata/SS/screenshot_16_084033.png
/kaggle/input/kavsakdata/SS/screenshot_16_213337.png
/kaggle/input/kavsakdata/SS/screenshot_16_204212.png
/kaggle/input/kavsakdata/SS/screenshot_16_083219.png
/kaggle/input/kavsakdata/SS/screenshot_16_051131.png
/kaggle/input/kavsakdata/SS/screenshot_16_065020.png
/kaggle/input/kavsakdata/SS/screenshot_16_105858.png
/kaggle/input/kavsakdata/SS/screenshot_16_093917.png
/kaggle/input/kavsakdata/SS/screenshot_16_173629.png
/kaggle/input/kavsakdata/SS/screenshot_16_0918

In [12]:
import cv2
import numpy as np
from datetime import datetime
import pandas as pd
from pathlib import Path  # ✅ Güvenli yol oluşturmak için

# ✅ Trafik renk kodları (güncel RGB tonları)
color_to_code = {
    (22, 224, 152): 1,
    (255, 207, 67): 2,
    (242, 78, 66): 3,
    (169, 39, 39): 4,
    (215, 223, 230): 0  # Gri - trafik yok
}

# ✅ En yakın trafik rengi kodunu döner
def get_closest_color_code(rgb_pixel):
    min_dist = float('inf')
    closest_code = 0
    for ref_rgb, code in color_to_code.items():
        dist = np.linalg.norm(np.array(rgb_pixel) - np.array(ref_rgb))
        if dist < min_dist:
            min_dist = dist
            closest_code = code
    return closest_code

# ✅ Ana analiz fonksiyonu
def analyze_image(image_path, csv_path="trafik_kayit.csv"):
    image_path = Path(image_path)
    image = cv2.imread(str(image_path))

    if image is None:
        print(f"❌ Görsel yüklenemedi: {image_path}")
        return

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Yol koordinatları
    yol_konumlari = {
        "Hatboyu-2 Cd.": [(1216, 436), (1365, 402), (1554, 359)],
        "Gazi Yakup Satar Cd. (Kuzey)": [(1181, 437), (1233, 340), (1308, 199)],
        "Gazi Yakup Satar Cd. (Guney)": [(1180, 508), (1163, 555), (1135, 637)],
        "Zubeyde Hanim Blv.": [(1138, 510), (1010, 541), (928, 559)],
        "Ihlamur Agaci Sk.": [(1196, 496), (1356, 467), (1523, 450)],
    }

    # ✅ Zamanı dosya adından al
    filename = image_path.stem  # 'screenshot_15_213958'
    try:
        parts = filename.split("_")
        if len(parts) >= 3:
            gun = int(parts[1])
            saat = int(parts[2][:2])
            dakika = int(parts[2][2:4])
            saniye = int(parts[2][4:6])
            zaman = f"2025-04-{gun:02d} {saat:02d}:{dakika:02d}:{saniye:02d}"
        else:
            raise ValueError("Dosya adı formatı hatalı")
    except Exception as e:
        print(f"⚠️ Zaman ayrıştırılamadı, sistem zamanı kullanılacak: {e}")
        zaman = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # ✅ Renkleri oku ve kaydet
    kayıtlar = []
    for yol_ismi, noktalar in yol_konumlari.items():
        renk_kodlari = []
        for (x, y) in noktalar:
            pixel = image_rgb[y, x]
            kod = get_closest_color_code(pixel)
            renk_kodlari.append(kod)
        kayıtlar.append([zaman, yol_ismi, *renk_kodlari])

    df = pd.DataFrame(kayıtlar, columns=["zaman", "yol_ismi", "bas_rengi", "orta_rengi", "son_rengi"])

    try:
        df.to_csv(csv_path, mode='a', header=not Path(csv_path).exists(), index=False)
        print(f"✅ CSV'ye kayıt eklendi: {image_path.name} → {zaman}")
    except Exception as e:
        print("CSV yazma hatası:", e)


# ✅ Kullanım
#analyze_image("/kaggle/input/kavsakdata/SS/screenshot_16_173608.png", csv_path="trafik_kayit2.csv")


In [13]:
import os

# Set the directory where your screenshots are stored
image_folder = "/kaggle/input/kavsakdata/SS"

# Loop through all files in the folder
for filename in sorted(os.listdir(image_folder)):
    image_path = os.path.join(image_folder, filename)
    analyze_image(image_path, csv_path="trafik_kayit_final.csv")


✅ CSV'ye kayıt eklendi: screenshot_15_213608.png → 2025-04-15 21:36:08
✅ CSV'ye kayıt eklendi: screenshot_15_213619.png → 2025-04-15 21:36:19
✅ CSV'ye kayıt eklendi: screenshot_15_213631.png → 2025-04-15 21:36:31
✅ CSV'ye kayıt eklendi: screenshot_15_213642.png → 2025-04-15 21:36:42
✅ CSV'ye kayıt eklendi: screenshot_15_213653.png → 2025-04-15 21:36:53
✅ CSV'ye kayıt eklendi: screenshot_15_213705.png → 2025-04-15 21:37:05
✅ CSV'ye kayıt eklendi: screenshot_15_213716.png → 2025-04-15 21:37:16
✅ CSV'ye kayıt eklendi: screenshot_15_213727.png → 2025-04-15 21:37:27
✅ CSV'ye kayıt eklendi: screenshot_15_213739.png → 2025-04-15 21:37:39
✅ CSV'ye kayıt eklendi: screenshot_15_213750.png → 2025-04-15 21:37:50
✅ CSV'ye kayıt eklendi: screenshot_15_213802.png → 2025-04-15 21:38:02
✅ CSV'ye kayıt eklendi: screenshot_15_213813.png → 2025-04-15 21:38:13
✅ CSV'ye kayıt eklendi: screenshot_15_213825.png → 2025-04-15 21:38:25
✅ CSV'ye kayıt eklendi: screenshot_15_213836.png → 2025-04-15 21:38:36
✅ CSV'